# CODIGO DE EJECUCION

*NOTA: Para poder usar este código de ejecución hay que lanzarlo desde exactamente el mismo entorno en el que fue creado.*

*Se puede instalar ese entorno en la nueva máquina usando el environment.yml que creamos en el set up del proyecto*

*Copiar el proyecto1.yml al directorio y en el terminal o anaconda prompt ejecutar:*

conda env create --file proyecto1.yml --name proyecto1

In [1]:
import cloudpickle
import pandas as pd
from janitor import clean_names

import warnings
warnings.filterwarnings("ignore")

ruta_proyecto = r'C:\Users\iagzg\Desktop\D4FB\D4FB\EstructuraDirectorio\03_MACHINE_LEARNING\06-MACHINE LEARNING PROJECT FRAMEWORK\CASOS\04_BANK\\'

nombre_fichero_datos = 'validacion.csv'

ruta_completa = ruta_proyecto + '/02_Datos/02_Validacion/' + nombre_fichero_datos

df = pd.read_csv(ruta_completa)

df.rename(columns = {'age':'edad',
                     'job':'trabajo',
                     'marital':'estado',
                     'education':'educacion',
                     'default':'impagos',
                     'housing':'prestamo_hipotecario',
                     'loan':'prestamo_personal',
                     'contact':'canal_contacto',
                     'month':'mes',
                     'day_of_week':'dia_semana',
                     'campaign':'num_contactos_esta_semana',
                     'pdays':'num_dias_ultimo_contacto',
                     'previous':'num_contactos_otras_campañas',
                     'poutcome':'resultado_campana_anterior',
                     'emp.var.rate':'variacion_tasa_empleo'
                    }, inplace=True) 
variables_finales = ['canal_contacto',
                     'dia_semana',
                     'edad',
                     'estado',
                     'mes',
                     'prestamo_hipotecario',
                     'trabajo',              
                     'variacion_tasa_empleo']

#Atipicos
num_desv_tip = 4

var_atipicos_dt = ['edad','variacion_tasa_empleo']

def atipicos_desv_tip(variable, num_desv_tip = 4):
    #sacamos los nulos por ahora
    variable = variable.dropna()
    #calculamos los límites
    media = np.mean(variable)
    sd = np.std(variable)
    umbral = sd * num_desv_tip
    lim_inf = media - umbral
    lim_sup = media + umbral
    #encontramos los índices de los que están fuera de los límites
    indices = [indice for indice,valor in variable.items() if valor < lim_inf or valor > lim_sup]
    return(indices)

def conteo_atipicos(df,variable, num_desv_tip = 4):
    atipicos = atipicos_desv_tip(df[variable], num_desv_tip)
    return(df.loc[atipicos,variable].value_counts())

for variable in var_atipicos_dt:
    df.drop(atipicos_desv_tip(df[variable], num_desv_tip),axis = 0, inplace = True)
    
df.drop('impagos', axis = 1, inplace = True)    

df = df[(df['prestamo_hipotecario'] != 'unknown') & (df['prestamo_personal'] != 'unknown')]    

                     
df = df[variables_finales]

nombre_pipe_ejecucion = 'pipe_ejecucion.pickle'

ruta_pipe_ejecucion = ruta_proyecto + '/04_Modelos/' + nombre_pipe_ejecucion

with open(ruta_pipe_ejecucion, mode='rb') as file:
   pipe_ejecucion = cloudpickle.load(file)

scoring = pipe_ejecucion.predict_proba(df)[:, 1]    